# Advanced Certification Program in Computational Data Science
## A program by IISc and TalentSprint
### Mini-Project: Speech Emotion Classification

## Problem Statement

Build a model to recognize emotion from speech using Ensemble learning 

## Learning Objectives

At the end of the mini-project, you will be able to :

* extract the features from audio data
* implement ML classification algorithms individually and as Ensembles, to classify emotions
* record the voice sample and test it with trained model

## Dataset

**TESS Dataset**

The first dataset chosen for this mini-project is the [TESS](https://dataverse.scholarsportal.info/dataset.xhtml?persistentId=doi:10.5683/SP2/E8H2MF) (Toronto emotional speech set) dataset. It contains 2880 files.  A set of 200 target words were spoken in the carrier phrase "Say the word _____' by two actresses and the sets were recorded in seven different emotions (anger, disgust, fear, happiness, pleasant surprise, sadness, and neutral). Both actresses spoke English as their first language, were university educated, and had musical training. Audiometric testing indicated that both actresses had thresholds within the normal range.

**Ravdess Dataset**

The second dataset chosen for this mini-project is [Ravdess](https://zenodo.org/record/1188976#.YLczy4XivIU) (The Ryerson Audio-Visual Database of Emotional Speech and Song). This dataset contains 1440 files: 60 trials per actor x 24 actors = 1440. The RAVDESS contains 24 professional actors (12 female, 12 male), vocalizing two lexically-matched statements in a neutral North American accent. Speech emotions includes calm, happy, sad, angry, fearful, surprise, and disgust expressions. Each expression is produced at two levels of emotional intensity (normal, strong), with an additional neutral expression.

**File naming convention**

Each of the 1440 files has a unique filename. The filename consists of a 7-part numerical identifier (e.g., 03-01-06-01-02-01-12.wav). These identifiers define the stimulus characteristics:

**Filename identifiers**

* Modality (01 = full-AV, 02 = video-only, 03 = audio-only).
* Vocal channel (01 = speech, 02 = song).
* Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
* Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.
* Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").
* Repetition (01 = 1st repetition, 02 = 2nd repetition).
* Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).

Filename example: `03-01-06-01-02-01-12.wav`

    - Audio-only - 03
    - Speech - 01
    - Fearful - 06
    - Normal intensity - 01
    - Statement "dogs" - 02
    - 1st Repetition - 01
    - 12th Actor - 12 Female, as the actor ID number is even.

## Information

**Speech Emotion Recognition (SER)** is the task of recognizing the emotion from  speech, irrespective of the semantics. Humans can efficiently perform this task as a natural part of speech communication, however, the ability to conduct it automatically using programmable devices is a field of active research.

Studies of automatic emotion recognition systems aim to create efficient, real-time methods of detecting the emotions of mobile phone users, call center operators and customers, car drivers, pilots, and many other human-machine communication users. Adding emotions to machines forms an important aspect of making machines appear and act in a human-like manner

Lets gain familiarity with some of the audio based features that are commonly used for SER. 

**Mel scale** — The mel scale (derived from the word *melody*) is a perceptual scale of pitches judged by listeners to be equal in distance from one another. The reference point between this scale and normal frequency measurement is defined by assigning a perceptual pitch of 1000 mels to a 1000 Hz tone, 40 dB above the listener's threshold. Above about 500 Hz, increasingly large intervals are judged by listeners to produce equal pitch increments. Refer [here](https://towardsdatascience.com/learning-from-audio-the-mel-scale-mel-spectrograms-and-mel-frequency-cepstral-coefficients-f5752b6324a8) for more detailed information.

**Pitch** — how high or low a sound is. It depends on frequency, higher pitch is high frequency

**Frequency** — speed of vibration of sound, measures wave cycles per second

**Chroma** — Representation for audio where spectrum is projected onto 12 bins representing the 12 distinct semitones (or chroma). Computed by summing the log frequency magnitude spectrum across octaves.

**Fourier Transforms** — used to convert from time domain to frequency domain. Time domain shows how signal changes over time. Frequency domain shows how much of the signal lies within each given frequency band over a range of frequencies

**Librosa**

[Librosa](https://librosa.org/doc/latest/index.html) is a Python package, built for speech and audio analytics. It provides modular functions that simplify working with audio data and help in achieving a wide range of applications such as identification of the personal characteristics of different individuals' voice samples, detecting emotions from audio samples etc. 

For further details on the Librosa package, refer [here](https://conference.scipy.org/proceedings/scipy2015/pdfs/brian_mcfee.pdf).


## Grading = 10 Points

### Download the dataset

In [ ]:
!wget -qq https://cdn.iisc.talentsprint.com/CDS/MiniProjects/M4_MP4_Ravdess_Tess.zip
!unzip -qq M4_MP4_Ravdess_Tess.zip
# Install packages
!pip -qq install librosa soundfile
!pip -qq install wavio

### Import Neccesary Packages

In [ ]:
import librosa
import librosa.display
import soundfile
import os, glob, pickle
import numpy as np
import pandas as pd
import IPython.display as ipd
from matplotlib import pyplot as plt
from datetime import datetime
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
import warnings
warnings.filterwarnings('ignore')
# sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import VotingClassifier

### Work-Flow

* Load the TESS audio data and extract features and labels

* Load the Ravdess audio data and extract features

* Combine both the audio dataset features

* Train and test the model with TESS + Ravdess Data

* Record the team audio samples and add them to TESS + Ravdess data

* Train and test the model with TESS + Ravdess + Team Recorded (combined) data

* Test each of the models with live audio sample recording.

### Load the Tess data and Ravdess data audio files (1 point)

Hint: `glob.glob`

In [ ]:
wav_files = glob.glob("Tess/*/*.wav")
len(wav_files)

#### Play the sample audio

In [ ]:
ipd.Audio('Tess/YAF_fear/YAF_cool_fear.wav')

### Data Exploration and Visualization (1 points)

#### Visualize the distribution of all the labels

In [ ]:
emotions_ = []
for file in wav_files:
    emotions_.append(file.split("_")[-1][:-4])
set(emotions_)

In [ ]:
from collections import Counter
freq_emotions = Counter(emotions_)
plt.bar(freq_emotions.keys(),freq_emotions.values())

#### Visualize sample audio signal using librosa

In [ ]:
sample_audio_path = 'Tess/YAF_fear/YAF_cool_fear.wav'
 
# librosa is used for analyzing and extracting features of an audio signal
data, sampling_rate = librosa.load(sample_audio_path)
plt.figure(figsize=(15, 5))
 
# librosa.display.waveplot is used to plot waveform of amplitude vs time
librosa.display.waveplot(data, sr=sampling_rate)
plt.show()

In [ ]:
sampling_rate

### Feature extraction (2 points)

Read one WAV file at a time using `Librosa`. An audio time series in the form of a 1-dimensional array for mono or 2-dimensional array for stereo, along with time sampling rate (which defines the length of the array), where the elements within each of the arrays represent the amplitude of the sound waves is returned by `librosa.load()` function. Refer to the supplementary notebook ('Audio feature extraction')

To know more about Librosa, explore the [link](https://librosa.org/doc/latest/feature.html)

In [ ]:
def extract_feature(file_name):
    """Function Extracts Features from WAV file"""
    X, sample_rate = librosa.load(file_name)
    stft=np.abs(librosa.stft(X))
    result=np.array([])
    mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    result=np.hstack((result, mfccs))
    chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    result=np.hstack((result, chroma))
    mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    result=np.hstack((result, mel))
    return result

In [ ]:
sample_feature = extract_feature(wav_files[0])#,mfcc=True, chroma=True, mel=True)
sample_feature.shape

#### Create a dictionary or a function to encode the emotions

In [ ]:
emotions = {'angry':5, 'disgust':7, 'fear':6, 'happy':3, 'neutral':1, 'surprised':8, 'sad':4}

#### TESS data feature extraction

**Note:** Features are already saved in DataFrame and given in csv file, Ignore the below commented code 

In [ ]:
# Load the data and extract features for each sound file
# tess_x, tess_y = [], []
# for file_name in wav_files:
#     emotion = emotions[file_name.split("_")[-1][:-4]]
#     feature = extract_feature(file_name)
#     tess_x.append(feature)
#     tess_y.append(emotion)
 
# len(tess_x), len(tess_y)

In [ ]:
# set(tess_y)

#### Ravdess data feature extraction

In [ ]:
rav_wav_files = glob.glob("ravdess/*/*.wav")
len(rav_wav_files), rav_wav_files[0]

In [ ]:
# rav_x, rav_y = [], []
# for file_name in rav_wav_files:
#     emotion = emotions[file_name.split("_")[-1][:-4]]
#     feature = extract_feature(file_name)
#     rav_x.append(feature)
#     rav_y.append(emotion)
# len(rav_x), len(rav_y)

In [ ]:
# set(rav_y)

In [ ]:
# features = tess_x + rav_x
# labels = tess_y + rav_y

In [ ]:
# Download extracted features csv
!wget -qq https://cdn.iisc.talentsprint.com/CDS/MiniProjects/tess_ravdess_features.csv

In [ ]:
extracted = pd.read_csv("tess_ravdess_features.csv")
features = extracted.iloc[:,:-1]
labels = extracted.iloc[:,-1]
features.shape, labels.shape

#### Save the features

It is best advised to save the features in dataframe and maintain so that feature extraction step is not required to be performed every time.

* Make a DataFrame with features and labels

* Write dataframe into `.CSV` file and save it offline.

In [ ]:
# saved_data = pd.DataFrame(features)
# saved_data['label'] = labels
# saved_data.to_csv("tess_ravdess_features.csv",index=False)

#### Split the data into train and test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(features), np.array(labels), test_size=0.2, random_state=42)
X_train.shape, X_test.shape, len(y_train)

### Train the model with TESS + Ravdess data (2 points)

* Apply different ML algorithms (eg. DecisionTree, RandomForest, etc.) and find the model with best performance

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)
lr.score(X_test, y_test), lr.score(X_train, y_train)

In [ ]:
# Decision Tree
from sklearn import tree
 
dt_model = tree.DecisionTreeClassifier(random_state=42)
dt_model = dt_model.fit(X_train, y_train)
dt_model.score(X_test, y_test), dt_model.score(X_train, y_train)

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
 
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
rf_model.score(X_test, y_test), rf_model.score(X_train, y_train)

In [ ]:
#Linear SVC
from sklearn.svm import LinearSVC

Lsvm = LinearSVC(random_state=0, tol=1e-5)
Lsvm.fit(X_train, y_train)
Lsvm.score(X_test, y_test)

In [ ]:
from sklearn.linear_model import SGDClassifier
 
sgd = SGDClassifier(max_iter=1000, tol=1e-3)

sgd.fit(X_train, y_train)
sgd.score(X_test, y_test)

#### Apply the voting classifier

In [ ]:
# Voting classifer
dt = tree.DecisionTreeClassifier(random_state=42)
lr = LogisticRegression(random_state=42)
sgd = SGDClassifier(max_iter=1000, tol=1e-3)
Lsvm = LinearSVC(random_state=0, tol=1e-5)
voting = VotingClassifier(estimators=[('lr',lr), ('dt', dt), ('sgd', sgd),('Lsvm',Lsvm)], voting='hard')
voting = voting.fit(X_train, y_train)
voting.score(X_test, y_test)

In [ ]:
!pip -qq install xgboost

In [ ]:
# xgb classifier
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
xgb.score(X_test, y_test)

#### Testing with a sample

In [ ]:
# audio sample 
test_sample = 'Tess/YAF_fear/YAF_cool_fear.wav'
ipd.Audio(test_sample)

In [ ]:
# prediction
pred = rf_model.predict(extract_feature(test_sample).reshape(1,-1))
list(emotions.keys())[list(emotions.values()).index(int(pred[0]))]

### Train the model with TESS + Ravdess + Team recorded data (2 points)

* Record the audio samples, extract features and combine with TESS data features
  - Record and gather all the team data with proper naming convention in separate folder

    **Hint:** Follow the supplementary notebook

  - Each team member must record 2 samples for each emotion (Use similar sentences as given in TESS data)

* Train the different ML algorithms and find the model with best performance

#### Load the team data

**Note:** Upload your team data by running the below cell and proceed with next cells

<font color="red">If the team data is not uploaded in the following cell, subsequent cells may result in errors.</font>

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!unzip -qq # Path here to unzip

**Change the path below**

In [ ]:
TeamData = glob.glob("/content/path_to_folder/*.wav")
len(TeamData)

#### Extracting features of team data and combine with TESS + Ravdess

In [ ]:
# extracting team recorded data features
team_features = []
team_label = []
for i in TeamData:
  team_features.append(extract_feature(i))
  emt = (i.split("_")[-1][:-4]).strip()
  team_label.append(emotions[emt])

len(team_features),len(team_label)

In [ ]:
# combining team data with Tess + ravdess data
all_x = list(features.values) + team_features
all_y = list(labels) + team_label
len(all_x), len(all_y)

In [ ]:
set(all_y)

#### Train the different ML algorithms

In [ ]:
# Split the data into train and test
xtrain, xtest, ytrain, ytest = train_test_split(np.array(all_x), np.array(all_y), test_size=0.2, random_state=42)
xtrain.shape, xtest.shape, len(ytrain)

In [ ]:
# Decision Tree
dt_model_team = tree.DecisionTreeClassifier(random_state=42)
dt_model_team = dt_model_team.fit(xtrain, ytrain)
dt_model_team.score(xtest, ytest), dt_model_team.score(xtrain, ytrain)

In [ ]:
# Random Forest
rf_model_team = RandomForestClassifier(random_state=42)
rf_model_team.fit(xtrain, ytrain)
rf_model_team.score(xtest, ytest), rf_model_team.score(xtrain, ytrain)

In [ ]:
#Linear SVC
Lsvm = LinearSVC(random_state=0, tol=1e-5)
Lsvm.fit(xtrain, ytrain)
Lsvm.score(xtest, ytest)

In [ ]:
# Voting classifer
dt = tree.DecisionTreeClassifier(random_state=42)
lr = LogisticRegression(random_state=42)
Lsvm = LinearSVC(random_state=0, tol=1e-5)
voting_team = VotingClassifier(estimators=[('lr',lr),('dt', dt),('Lsvm',Lsvm)], voting='hard')
voting_team = voting_team.fit(xtrain, ytrain)
voting_team.score(xtest, ytest)

In [ ]:
# xgb classifier
xgb = XGBClassifier()
xgb.fit(xtrain, ytrain)
xgb.score(xtest, ytest)

### Test the best working model with live audio recording

In [ ]:
MODEL = rf_model

In [ ]:
#@title Speak the utterance and test
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
 
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

if not os.path.exists('ModelTesting/'):
    os.mkdir("ModelTesting/")
def record(sec=3):
    print("Start speaking!")
    now = datetime.now()
    current_time = now.strftime("%Y-%m-%d_%H-%M-%S")
    display(Javascript(RECORD))
    s = output.eval_js('record(%d)' % (sec*1000))
    b = b64decode(s.split(',')[1])
    with open('ModelTesting/audio_'+current_time+'.wav','wb') as f:
        f.write(b)
    return 'ModelTesting/audio_'+current_time+'.wav'
test_i = record()
pred = MODEL.predict(extract_feature(test_i).reshape(1,-1))
idx_emotion = list(emotions.values()).index(pred[0])
print(list(emotions.keys())[idx_emotion])
ipd.Audio(test_i)

### Report Analysis

- Report the accuracy for 10 live samples using the model trained on TESS+Ravdess+Team data
- Discuss with the team mentor regarding deep learnt audio features (which will be introduced in Module 5 for another audio classification task). Read a related article [here](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8805181).
